In [205]:
import pandas as pd
import numpy as np
import math
from pathlib import Path

In [206]:
BASE = Path("Datasets/mockup_ver2/")

products   = pd.read_csv(BASE/"products.csv")
promotions = pd.read_csv(BASE/"promotions.csv", parse_dates=["start_date","end_date"])
promo_tx   = pd.read_csv(BASE/"promotion_transactions.csv")
stores     = pd.read_csv(BASE/"stores_with_patterns.csv")
tx         = pd.read_csv(BASE/"transactions.csv", parse_dates=["timestamp"])
users      = pd.read_csv(BASE/"users_features_with_segments.csv")  # เผื่อไว้ แม้จะมีแค่ user_id
festivals  = pd.read_csv(BASE/"festivals_2025.csv")

tx_merge = pd.read_csv(BASE/"tx_merge1.csv")  # from previous notebook

C:\Users\User\AppData\Local\Temp\ipykernel_28604\63410452.py:4: UserWarning: Parsing dates in %d/%m/%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  promotions = pd.read_csv(BASE/"promotions.csv", parse_dates=["start_date","end_date"])
C:\Users\User\AppData\Local\Temp\ipykernel_28604\63410452.py:4: UserWarning: Parsing dates in %d/%m/%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  promotions = pd.read_csv(BASE/"promotions.csv", parse_dates=["start_date","end_date"])
C:\Users\User\AppData\Local\Temp\ipykernel_28604\63410452.py:7: UserWarning: Parsing dates in %d/%m/%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  tx         = pd.read_csv(BASE/"transactions.csv", parse_dates=["timestamp"])


In [207]:
tx_merge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19178 entries, 0 to 19177
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   transaction_id       19178 non-null  object
 1   user_id              19178 non-null  object
 2   product_id           19178 non-null  object
 3   qty                  19178 non-null  int64 
 4   price                19178 non-null  int64 
 5   timestamp            19178 non-null  object
 6   store_id             19178 non-null  object
 7   is_online            19178 non-null  bool  
 8   products.category    19178 non-null  object
 9   products.brand       19178 non-null  object
 10  products.base_price  19178 non-null  int64 
 11  stores.zone          19178 non-null  int64 
 12  stores.province      19178 non-null  object
 13  stores.profile       19178 non-null  object
 14  promo_id             5076 non-null   object
dtypes: bool(1), int64(4), object(10)
memory usage: 2.1+ M

## Time Feature

In [208]:
tx_merge["timestamp"] = pd.to_datetime(
    tx_merge["timestamp"], 
    format="%d/%m/%Y %H:%M"   # รูปแบบวันที่-เวลาในไฟล์
)

tx_merge["order_date"] = tx_merge["timestamp"].dt.date
tx_merge["order_hour"] = tx_merge["timestamp"].dt.hour
tx_merge["dayofweek"] = tx_merge["timestamp"].dt.dayofweek
tx_merge["month"] = tx_merge["timestamp"].dt.month
tx_merge["day"] = tx_merge["timestamp"].dt.day
tx_merge["weekofyear"] = tx_merge["timestamp"].dt.isocalendar().week
tx_merge["quarter"] = tx_merge["timestamp"].dt.quarter
tx_merge["is_weekend"] = tx_merge["dayofweek"].apply(lambda x: 1 if x >= 5 else 0)

tx_merge["store_id"] = tx_merge["store_id"].astype(str).str.strip()

def month_to_th_season(m):
    if pd.isna(m):
        return np.nan
    m = int(m)
    if m in (3,4,5):
        return "Summer"
    elif m in (6,7,8,9,10):
        return "Rainy"
    else:  # 11,12,1,2
        return "Winter"

tx_merge["thai_season"] = tx_merge["month"].apply(month_to_th_season)

stores["store_id"] = stores["store_id"].astype(str).str.strip()


In [209]:
tx_merge.isna().sum()

transaction_id             0
user_id                    0
product_id                 0
qty                        0
price                      0
timestamp                  0
store_id                   0
is_online                  0
products.category          0
products.brand             0
products.base_price        0
stores.zone                0
stores.province            0
stores.profile             0
promo_id               14102
order_date                 0
order_hour                 0
dayofweek                  0
month                      0
day                        0
weekofyear                 0
quarter                    0
is_weekend                 0
thai_season                0
dtype: int64

In [210]:
festivals = festivals.rename(columns={"category":"festival_category"})

for c in ["start_date","end_date"]:
    festivals[c] = (festivals[c].astype(str)
                   .str.replace("\u200b","", regex=False)  # zero-width space
                   .str.strip()
                   .replace({"": None, "NULL": None, "NaN": None}))
    
    
festivals["start_date"] = pd.to_datetime(festivals["start_date"], errors="coerce")
festivals["end_date"]   = pd.to_datetime(festivals["end_date"], errors="coerce")
festivals["date"] = [pd.date_range(s.normalize(), e.normalize(), freq="D") for s, e in zip(festivals["start_date"], festivals["end_date"])]
fest_daily = festivals.explode("date", ignore_index=True)
fest_daily["date"] = fest_daily["date"].dt.date
festivals.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29 entries, 0 to 28
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   festival_name      29 non-null     object        
 1   start_date         29 non-null     datetime64[ns]
 2   end_date           29 non-null     datetime64[ns]
 3   festival_category  29 non-null     object        
 4   date               29 non-null     object        
dtypes: datetime64[ns](2), object(3)
memory usage: 1.3+ KB


In [211]:
# ทำคีย์ให้ตรงกัน
tx_merge["order_date_norm"] = pd.to_datetime(tx_merge["order_date"], errors="coerce").dt.normalize()

fest = fest_daily.copy()
if "festival_category" not in fest.columns and "category" in fest.columns:
    fest = fest.rename(columns={"category": "festival_category"})

fest["date_norm"] = pd.to_datetime(fest["date"], errors="coerce").dt.normalize()

# รวมให้เหลือ 1 แถว/วัน (ถ้าวันเดียวมีหลายเทศกาลจะรวมด้วย "; ")
fest_by_day = (
    fest.groupby("date_norm", as_index=False)
        .agg(
            festival_name=("festival_name",  lambda s: "; ".join(sorted(set(s.dropna().astype(str).str.strip())))),
            festival_category=("festival_category", lambda s: "; ".join(sorted(set(s.dropna().astype(str).str.strip())))),
        )
)

# merge (many-to-one หลังสรุปแล้ว)
tx_merge = tx_merge.merge(
    fest_by_day, left_on="order_date_norm", right_on="date_norm",
    how="left", validate="m:1"
)

# clean + flag
tx_merge["festival_name"]     = tx_merge["festival_name"].fillna("").astype(str).str.strip()
tx_merge["festival_category"] = tx_merge["festival_category"].fillna("").astype(str).str.strip()
tx_merge["InFestival"]        = (tx_merge["festival_name"] != "").astype(int)

# เก็บตารางให้สะอาด
tx_merge = tx_merge.drop(columns=["order_date_norm","date_norm"])


In [212]:
tx_merge.isna().sum()

transaction_id             0
user_id                    0
product_id                 0
qty                        0
price                      0
timestamp                  0
store_id                   0
is_online                  0
products.category          0
products.brand             0
products.base_price        0
stores.zone                0
stores.province            0
stores.profile             0
promo_id               14102
order_date                 0
order_hour                 0
dayofweek                  0
month                      0
day                        0
weekofyear                 0
quarter                    0
is_weekend                 0
thai_season                0
festival_name              0
festival_category          0
InFestival                 0
dtype: int64

In [213]:
tx_merge.head()

,transaction_id,user_id,product_id,qty,price,timestamp,store_id,is_online,products.category,products.brand,...,dayofweek,month,day,weekofyear,quarter,is_weekend,thai_season,festival_name,festival_category,InFestival
0,TX0000510,U0038,P0442,4,694,2025-09-22 01:44:00,S042,False,Others,Brand_001,...,0,9,22,39,3,0,Rainy,เทศกาลกินเจ (Vegetarian Festival),เทศกาลไทย-จีน,1
1,TX0000764,U0058,P0689,4,514,2025-09-21 20:21:00,S042,False,HealthBeauty,Brand_032,...,6,9,21,38,3,1,Rainy,,,0
2,TX0001136,U0086,P0078,4,460,2025-06-29 04:36:00,S042,False,DairyBakery,Brand_002,...,6,6,29,26,2,1,Rainy,,,0
3,TX0001608,U0117,P0653,4,390,2025-06-25 23:37:00,S042,False,Household,Brand_012,...,2,6,25,26,2,0,Rainy,,,0
4,TX0001699,U0125,P0843,4,462,2025-08-04 01:28:00,S042,False,HealthBeauty,Brand_006,...,0,8,4,32,3,0,Rainy,,,0


Store Time Features

In [214]:
hour_cols = [c for c in stores.columns if c.startswith("hour_w_")]

st_hour_long = stores.melt(
    id_vars=["store_id", "profile", "weekday_boost", "weekend_boost", "festival_boost", "peaks_encoded"],
    value_vars=hour_cols, var_name="hour_col", value_name="hour_weight"
)

st_hour_long["order_hour"] = st_hour_long["hour_col"].str.replace("hour_w_", "", regex=False).astype(int)
st_hour_long = st_hour_long.drop(columns=["hour_col"])

tx_merge = tx_merge.merge(
    st_hour_long,
    on=["store_id", "order_hour"],
    how="left",
    validate="m:1"
)

In [215]:
tx_merge.columns

Index(['transaction_id', 'user_id', 'product_id', 'qty', 'price', 'timestamp',
       'store_id', 'is_online', 'products.category', 'products.brand',
       'products.base_price', 'stores.zone', 'stores.province',
       'stores.profile', 'promo_id', 'order_date', 'order_hour', 'dayofweek',
       'month', 'day', 'weekofyear', 'quarter', 'is_weekend', 'thai_season',
       'festival_name', 'festival_category', 'InFestival', 'profile',
       'weekday_boost', 'weekend_boost', 'festival_boost', 'peaks_encoded',
       'hour_weight'],
      dtype='object')

### User


In [216]:
users = users.drop(columns=["segment_desc"])

In [217]:
tx_merge= tx_merge.merge(users , on="user_id", how="left", validate="m:1")

In [218]:
if "promo_id" in tx_merge.columns:
    # กรณี promo_id เป็นตัวเลข: นับเป็นโปรเมื่อไม่ใช่ NaN และไม่ใช่ 0
    if pd.api.types.is_numeric_dtype(tx_merge["promo_id"]):
        tx_merge["has_promotion"] = (
            tx_merge["promo_id"].notna() & (tx_merge["promo_id"] != 0)
        ).astype(int)
    else:
        # กรณีเป็นสตริง/ผสม: นับเป็นโปรเมื่อไม่ว่าง และไม่ใช่ "0"
        tx_merge["has_promotion"] = (
            tx_merge["promo_id"]
            .astype(str)
            .str.strip()
            .str.lower()
            .replace({"nan": "", "none": "", "null": ""})
            .pipe(lambda s: (s != "") & (s != "0"))
        ).astype(int)
else:
    # ถ้าไม่มี promo_id ให้สร้างคอลัมน์ไว้ก่อน (ไม่มีโปรทั้งหมด)
    tx_merge["has_promotion"] = 0

In [219]:
tx_merge.tail(20) 

,transaction_id,user_id,product_id,qty,price,timestamp,store_id,is_online,products.category,products.brand,...,weekday_boost,weekend_boost,festival_boost,peaks_encoded,hour_weight,loyalty_score,expected_basket_items,price_elasticity,segment,has_promotion
19158,PMTX0005002,U0846,P0692,3,229,2025-07-16 21:58:00,S003,False,ReadyToEat,Brand_037,...,1.0,1.05,1.05,07-09;17-21,0.888889,0.958,3.00,-0.32,2,1
19159,PMTX0005041,U0851,P0114,3,35,2025-08-13 18:02:00,S003,False,Snacks,Brand_043,...,1.0,1.05,1.05,07-09;17-21,1.333333,0.921,3.00,-0.15,2,1
19160,PMTX0005064,U0855,P0704,4,75,2025-07-24 20:27:00,S003,False,Beverages,Brand_048,...,1.0,1.05,1.05,07-09;17-21,1.333333,0.919,2.99,-0.15,3,1
19161,PMTX0005192,U0875,P0962,2,227,2025-08-21 06:56:00,S003,False,HealthBeauty,Brand_010,...,1.0,1.05,1.05,07-09;17-21,0.888889,0.916,2.98,-0.05,3,1
19162,PMTX0005263,U0887,P0277,1,48,2025-07-26 03:39:00,S003,False,PersonalCare,Brand_018,...,1.0,1.05,1.05,07-09;17-21,0.888889,0.902,3.01,-0.01,1,1
19163,PMTX0005410,U0908,P0441,2,53,2025-08-10 06:23:00,S003,False,Snacks,Brand_008,...,1.0,1.05,1.05,07-09;17-21,0.888889,0.919,3.00,-0.19,2,1
19164,PMTX0005439,U0914,P0422,1,138,2025-07-17 15:56:00,S003,False,FrozenFoods,Brand_036,...,1.0,1.05,1.05,07-09;17-21,0.888889,0.939,2.99,0.23,1,1
19165,PMTX0005441,U0914,P0322,3,60,2025-08-24 13:13:00,S003,False,Household,Brand_040,...,1.0,1.05,1.05,07-09;17-21,0.888889,0.939,2.99,0.23,1,1
19166,PMTX0005449,U0915,P0097,1,39,2025-08-22 14:26:00,S003,False,Snacks,Brand_031,...,1.0,1.05,1.05,07-09;17-21,0.888889,0.902,2.97,0.24,4,1
19167,PMTX0005606,U0937,P0546,3,178,2025-08-13 18:27:00,S003,False,HealthBeauty,Brand_004,...,1.0,1.05,1.05,07-09;17-21,1.333333,0.935,2.99,-0.50,2,1


In [224]:
tx_merge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19178 entries, 0 to 19177
Data columns (total 38 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   transaction_id         19178 non-null  object        
 1   user_id                19178 non-null  object        
 2   product_id             19178 non-null  object        
 3   qty                    19178 non-null  int64         
 4   price                  19178 non-null  int64         
 5   timestamp              19178 non-null  datetime64[ns]
 6   store_id               19178 non-null  object        
 7   is_online              19178 non-null  bool          
 8   products.category      19178 non-null  object        
 9   products.brand         19178 non-null  object        
 10  products.base_price    19178 non-null  int64         
 11  stores.zone            19178 non-null  int64         
 12  stores.province        19178 non-null  object        
 13  s

In [227]:
tx_merge.to_csv(BASE/"tx_merge2.csv", index=False)
tx_merge.to_excel(BASE/"tx_merge2.xlsx", index=False)
